In [2]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import *

from src.utilities import *
from src.bubble import *

import openai
import instructor

Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [3]:
prompts_path = "Prompts/fr/"

# Company informations

In [5]:
with open(prompts_path+'prompt_company_informations.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_company_informations = template.invoke(context)

class CompanyContext(BaseModel):
    context: str = Field(description="Description du contexte de l'entreprise.")

    def __str__(self):
        return self.context

res = get_analysis(prompt_company_informations, CompanyContext)
context['context'] = res.context
print(res)

Darty est une chaîne de magasins française spécialisée dans la distribution de matériel électroménager, de produits informatiques et audiovisuels (TV, audio, photo). Fondée en 1957, l'entreprise a évolué au fil des années, devenant un acteur majeur dans son secteur en France. Récemment, Darty a continué à se développer en renforçant sa présence en ligne et en élargissant son offre de services, notamment avec des options de livraison améliorées et des services après-vente étendus. Elle fait face à une concurrence accrue de la part de grandes enseignes telles que Boulanger, Fnac, et des géants du e-commerce comme Amazon. Darty s'efforce de maintenir sa position sur le marché en mettant l'accent sur la qualité de service et l'expérience client.


# Categories creation

In [18]:
with open(prompts_path+'prompt_categories_creation.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_categories_creation = template.invoke(context)

class InsightSubCategory(BaseModel):
    name: str = Field(description="Nom de la sous-catégorie.")
    #description: str = Field(description="Phrase de description de la sous-catégorie.")
    id: str
    def __str__(self):
        return '\n- ' + self.name #+ ": " + self.description

class InsightCategory(BaseModel):
    name: str = Field(description="Nom de la catégorie.")
    #description: str = Field(description="Phrase de description de la catégorie.")
    sub_categories: List[InsightSubCategory] = Field(description="Sous-catégories.")
    id: str

    def __str__(self):
        return self.name + " : " + ''.join([str(x) for x in self.sub_categories])

class InsightCategories(BaseModel):
    categories: List[InsightCategory] = Field(description="Liste des catégories parmi lesquels les insights pourront être classifiées.")

    def __str__(self):
        return '\n\n'.join([str(x) for x in self.categories])

res = get_analysis(prompt_categories_creation, InsightCategories)
print(res)

Service Client : 
- Accueil et assistance
- Temps d'attente
- Résolution de problèmes
- Communication

Produits : 
- Qualité
- Disponibilité
- Diversité
- Prix

Expérience en Magasin : 
- Aménagement
- Propreté
- Accessibilité
- Sécurité

Site Web et Application : 
- Facilité d'utilisation
- Fonctionnalités
- Performance
- Sécurité

Livraison et Installation : 
- Rapidité
- Fiabilité
- Coût
- Service après-vente

Politique de Retour : 
- Facilité
- Rapidité
- Conditions
- Remboursement


In [ ]:
#res.categories.append(InsightCategory(name=""))
#for categ in res.categories:
#    categ.append(sub_categories=InsightSubCategory(name=""))

In [22]:
for categorie in tqdm(res.categories):
    categorie.id = bubble_client.create(
        "Category",
        {
            "Company": COMPANY_ID,
            "Name": categorie.name,
            "Project": PROJECT_ID,
        })
    
    for sub_categorie in categorie.sub_categories:
        sub_categorie.id = bubble_client.create(
            "SubCategory",
            {
                "Company": COMPANY_ID,
                "Category": categorie.id,
                "Name": sub_categorie.name,
                "Project": PROJECT_ID,
                #"Description": sub_categorie.description,
            })

  0%|          | 0/6 [00:00<?, ?it/s]